# DSC 180B CNN Notebook

### Data Loading and Getting the Labels

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import torch
import cv2
from imgaug import augmenters as iaa
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
from torchvision import models
import torch.nn as nn

In [ ]:
df = pd.read_csv('../../data/sql-data/Data_Entry_2017.csv')
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
def get_label(row):
    if 'No Finding' in row['Finding Labels']:
        return 0
    return 1

In [ ]:
df['Finding Labels'] = df.apply(lambda row: get_label(row), axis=1)
df.head()

In [ ]:
def transform_image(img):
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img  

def random_horizontal_flip(img):
    if np.random.rand() < 0.5:
        img = cv2.flip(img, 1)
    return img

def apply_clahe(img):
    red_channel = img[:, :, 0]
    green_channel = img[:, :, 1]
    blue_channel = img[:, :, 2]
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    
    red_channel = clahe.apply(red_channel)
    green_channel = clahe.apply(green_channel)
    blue_channel = clahe.apply(blue_channel)
    
    return cv2.merge((red_channel, green_channel, blue_channel))

def add_random_rotation(img, max_angle=10):
    angle = np.random.uniform(-max_angle, max_angle)
    M = cv2.getRotationMatrix2D((img.shape[1] // 2, img.shape[0] // 2), angle, 1)
    return cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

def normalize_image(img):
    return img / 255.0

In [ ]:
X = []
y = []

# load png images into X and y
for i in range(len(df)):
    image_name = df['Image Index'][i]
    png_path = os.path.join('../../data/raw', image_name) # Use os.path.join to create the path

    # Check if the file exists before trying to read it
    if not os.path.exists(png_path):
        # print(f"Image file not found: {png_path}")
        continue

    # print("Image found")

    img = cv2.imread(png_path)

    # Check if the image was loaded successfully
    if img is None:
        # print(f"Failed to load image: {png_path}")
        continue

    img = transform_image(img)
    img = random_horizontal_flip(img)
    img = apply_clahe(img)
    img = add_random_rotation(img)
    img = normalize_image(img)

    X.append(img)
    y.append(df['Finding Labels'][i])

    # print(len(X))
    # print(len(y))

X = np.array(X)
y = np.array(y)

In [ ]:
# # Convert to tensor to be fed to ResNet50
# X = torch.tensor(X, dtype=torch.float32)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Healthy vs Unhealthy Lung Comparison

In [ ]:
healthy_x_rays = []
abnormal_x_rays = []
i = 0
while len(healthy_x_rays) < 10 or len(abnormal_x_rays) < 10:
    if y_train[i] == 0:
        healthy_x_rays.append(X_train[i])
    else:
        abnormal_x_rays.append(X_train[i])
    i += 1

for i in range(10):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(healthy_x_rays[i], cmap='gray')
    ax[0].set_title('Healthy X-Ray')
    ax[1].imshow(abnormal_x_rays[i], cmap='gray')
    ax[1].set_title('Abnormal X-Ray')
    plt.show()


In [64]:
import numpy as np
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts

# Compute class weights
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Load ResNet50V2
resnet50 = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Unfreeze deeper layers for fine-tuning
for layer in resnet50.layers:
    layer.trainable = False

# Define the model
model = Sequential([
    resnet50,
    Flatten(),
    Dense(1, activation="sigmoid")
])

# Implement Cosine Annealing
initial_lr = 1e-4
cosine_decay_restarts = CosineDecayRestarts(initial_learning_rate=initial_lr,
                                            first_decay_steps=5000,
                                            t_mul=2.0,  # Increases period after each restart
                                            m_mul=0.8,  # Reduce max learning rate after each restart
                                            alpha=1e-6)

# Compile model with Cosine Annealing scheduler
model.compile(optimizer=Adam(learning_rate=cosine_decay_restarts),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    batch_size=16,
                    class_weight=class_weights_dict,
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
5/5 [==============================] - ETA: 0s - loss: 1.1373 - accuracy: 0.5500
Epoch 1: val_loss improved from inf to 1.65515, saving model to best_model.keras
5/5 [==============================] - 6s 543ms/step - loss: 1.1373 - accuracy: 0.5500 - val_loss: 1.6552 - val_accuracy: 0.3500
Epoch 2/100
5/5 [==============================] - ETA: 0s - loss: 0.4250 - accuracy: 0.8125
Epoch 2: val_loss improved from 1.65515 to 0.87390, saving model to best_model.keras
5/5 [==============================] - 1s 287ms/step - loss: 0.4250 - accuracy: 0.8125 - val_loss: 0.8739 - val_accuracy: 0.5000
Epoch 3/100
5/5 [==============================] - ETA: 0s - loss: 0.2286 - accuracy: 0.9125
Epoch 3: val_loss did not improve from 0.87390
5/5 [==============================] - 0s 57ms/step - loss: 0.2286 - accuracy: 0.9125 - val_loss: 0.9634 - val_accuracy: 0.5500
Epoch 4/100
5/5 [==============================] - ETA: 0s - loss: 0.0914 - accuracy: 0.9875
Epoch 4: val_loss did not imp

In [ ]:
# create a confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(X_test)
y_pred = y_pred > 0.5
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

# output precision, recall, f1-score
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



In [ ]:
# prompt: Save the .keras and .h5 file to my google drive

# Save the model to your Google Drive
model.save('./best_model.h5')
model.save('./best_model.keras')
